In [1]:
from dotenv import load_dotenv
from crewai import Crew
from tasks import MemoryTasks
from agents import LTMAgents
from memo_store import MemoStore
load_dotenv()

True

In [2]:
# memo_store.reset_db()

In [3]:
memo_store = MemoStore(verbosity=0, reset=False, path_to_db_dir="./tmp/teachable_agent_db")


LOADING MEMORY FROM DISK
    Location = ./tmp/teachable_agent_db/uid_text_dict.pkl


In [4]:
user_context = "I want to create a meal plan for myself for lunch but Im allergic to fish?"

In [5]:

tasks = MemoryTasks()
agents = LTMAgents()

# create agents
context_agent = agents.context_agent()
memory_agent = agents.memory_agent()

# create tasks
context_task = tasks.determine_memory_task(context_agent, user_context)
memory_task = tasks.store_memory(memory_agent, memo_store)

memory_task.context = [context_task]
    
crew = Crew(
    agents=[
        context_agent, 
        memory_agent
    ],
    tasks=[
        context_task,
        memory_task
    ]
)

In [6]:
result = crew.kickoff()



> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: I'm allergic to fish.

> Finished chain.


> Entering new CrewAgentExecutor chain...

In [7]:
memo_store.list_memos()

LIST OF MEMOS
